# Playground SQL

Pour rappel, on a 4 tables suivantes:

- Athlete : id (primary key), name, gender, team
- Event : id (primary key), location, year, distance, stroke, relay
- Team : id (primary key), event_id (foreign key), athlete_id (foreign key)
- Results : id (primary key), team_id (foreign key), results, rank, quit_reason

Le but ici va être de s'entraîner sur des requêtes SQL avancées.

In [1]:
# Connexion à la base de données
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")

# Création de la base de données
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')


GPT o1-mini, avec la connaissance du contexte, nous a concocté quelques exercices :

Voici une série de défis SQL conçus pour vous aider à vous entraîner et à maîtriser l'utilisation des Common Table Expressions (CTEs), des fonctions de fenêtre (window functions) et des classements (RANKs) dans le contexte de votre base de données sur les résultats de natation olympique.

1. Top 3 Performers par Année et Style de Nage

Écrivez une requête SQL utilisant un CTE et des fonctions de fenêtre pour identifier les trois athlètes les plus performants (les résultats les plus bas) dans chaque style de nage pour chaque année olympique.

2. Moyenne des Résultats par Équipe

Utilisez une fonction de fenêtre pour calculer le temps moyen des résultats pour chaque équipe à travers tous les événements et listez les équipes dont la performance est supérieure à la moyenne générale.

3. Amélioration des Performances au Fil du Temps

Employez un CTE pour organiser les données par athlète et année, puis utilisez des fonctions de fenêtre pour calculer la différence de résultats par rapport à l'année olympique précédente pour chaque athlète.

4. Classement de Participation aux Événements

Classez les athlètes en fonction du nombre d'événements auxquels ils ont participé en utilisant RANK ou DENSE_RANK, et listez les dix athlètes les plus actifs.

5. Performance des Équipes de Relais

Utilisez des fonctions de fenêtre pour déterminer quelles équipes de relais ont les résultats combinés les plus rapides et classez-les en conséquence.

6. Résultats Médians par Événement

Écrivez une requête utilisant des fonctions de fenêtre pour calculer le temps médian des résultats pour chaque événement et identifiez comment chaque athlète se compare à la médiane.

7. Consistance des Performances des Athlètes

Utilisez des CTE et des fonctions de fenêtre pour mesurer la consistance de chaque athlète en calculant l'écart type de leurs temps de résultats à travers différents événements, et classez-les du plus au moins constant.

8. Comparaison des Performances par Genre et Année

Utilisez des fonctions de fenêtre pour comparer les résultats moyens entre les athlètes masculins et féminins pour chaque année, et classez l'évolution des performances par genre au fil du temps.

9. Domination des Équipes par Style de Nage

Utilisez RANK pour identifier quelles équipes ont remporté le plus de médailles dans chaque catégorie de style de nage à travers toutes les années.

10. Plus Grande Amélioration des Résultats

Utilisez des CTE pour identifier les athlètes qui ont montré la plus grande amélioration dans leurs résultats au fil des Jeux Olympiques successifs.

---
Ces défis vous permettront de manipuler et d'analyser vos données de manière approfondie tout en renforçant vos compétences en SQL avancé. Bon entraînement !

## Exercice 1 : Top 3 Performers par Année et Style de Nage

"Écrivez une requête SQL utilisant un CTE et des fonctions de fenêtre pour identifier les trois athlètes les plus performants (les résultats les plus bas) dans chaque style de nage pour chaque année olympique."

On va pas compter les relais en revanche, parce qu'on a pas l'info de qui dans l'équipe a été meilleur dans le dataset.

On va aussi prendre en compte la distance, ça n'a pas de sens de comparer les résultats UNIQUEMENT par style de nage.

Voici le résultat en prenant en compte toutes les données, pas besoin de CTE ou de window function c'est trivial.

In [18]:
from sqlalchemy import text

session = engine.connect()

query = text("""
SELECT a.name, e.stroke, e.distance, e.year, r.rank
FROM athletes a
INNER JOIN event_teams t ON a.athlete_id = t.athlete_id
INNER JOIN events e ON t.event_id = e.event_id
INNER JOIN results r ON t.event_team_id = r.event_team_id
WHERE NOT e.is_relay
AND r.rank <= 3
AND r.rank > 0
ORDER BY e.year, e.stroke, e.distance, r.rank
""")

result = session.execute(query)
print(result.fetchall())

session.close()

[('Harry J. Hebner', 'Backstroke', 100, 1912, 1), ('Otto Fahr', 'Backstroke', 100, 1912, 2), ('Paul Kellner', 'Backstroke', 100, 1912, 3), ('Walter Bathe', 'Breaststroke', 200, 1912, 1), ('Willy Lützow', 'Breaststroke', 200, 1912, 2), ('Kurt Paul Malisch', 'Breaststroke', 200, 1912, 3), ('Walter Bathe', 'Breaststroke', 400, 1912, 1), ('Tor Henning', 'Breaststroke', 400, 1912, 2), ('Percy Courtman', 'Breaststroke', 400, 1912, 3), ('Fanny Durack', 'Freestyle', 100, 1912, 1), ('Duke Paoa Kahanamoku', 'Freestyle', 100, 1912, 1), ('Cecil Healy', 'Freestyle', 100, 1912, 2), ('Mina Wylie', 'Freestyle', 100, 1912, 2), ('Jennie Fletcher', 'Freestyle', 100, 1912, 3), ('Kenneth Huszagh', 'Freestyle', 100, 1912, 3), ('George Ritchie Hodgson', 'Freestyle', 400, 1912, 1), ('John Gatenby Hatfield', 'Freestyle', 400, 1912, 2), ('Harold H. Hardwick', 'Freestyle', 400, 1912, 3), ('George Ritchie Hodgson', 'Freestyle', 1500, 1912, 1), ('John Gatenby Hatfield', 'Freestyle', 1500, 1912, 2), ('Harold H. Har

On va maintenant oublier que le rank a déjà été calculé et calculer cela nous-mêmes, on prendra quand même soin d'éliminer les tuples qui ont été disqualifiés pour prévenir tout résultat biaisé.

In [19]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH QualifiedSoloAthletes AS (
    SELECT
        a.athlete_id, e.stroke, e.distance, e.year, a.name, r.results,
        RANK() OVER (PARTITION BY year, stroke, distance ORDER BY results ASC) AS rank
    FROM athletes a
    INNER JOIN event_teams t ON a.athlete_id = t.athlete_id
    INNER JOIN events e ON t.event_id = e.event_id
    INNER JOIN results r ON t.event_team_id = r.event_team_id
    WHERE NOT e.is_relay
    AND r.rank > 0
)
SELECT 
    name, stroke, distance, year, rank
FROM QualifiedSoloAthletes
WHERE rank <= 3
ORDER BY year, stroke, distance, rank
""")

result = session.execute(query)
print(result.fetchall())

session.close()

[('Harry J. Hebner', 'Backstroke', 100, 1912, 1), ('Otto Fahr', 'Backstroke', 100, 1912, 2), ('Paul Kellner', 'Backstroke', 100, 1912, 3), ('Walter Bathe', 'Breaststroke', 200, 1912, 1), ('Willy Lützow', 'Breaststroke', 200, 1912, 2), ('Kurt Paul Malisch', 'Breaststroke', 200, 1912, 3), ('Walter Bathe', 'Breaststroke', 400, 1912, 1), ('Tor Henning', 'Breaststroke', 400, 1912, 2), ('Percy Courtman', 'Breaststroke', 400, 1912, 3), ('Duke Paoa Kahanamoku', 'Freestyle', 100, 1912, 1), ('Cecil Healy', 'Freestyle', 100, 1912, 2), ('Kenneth Huszagh', 'Freestyle', 100, 1912, 3), ('George Ritchie Hodgson', 'Freestyle', 400, 1912, 1), ('John Gatenby Hatfield', 'Freestyle', 400, 1912, 2), ('Harold H. Hardwick', 'Freestyle', 400, 1912, 3), ('George Ritchie Hodgson', 'Freestyle', 1500, 1912, 1), ('John Gatenby Hatfield', 'Freestyle', 1500, 1912, 2), ('Harold H. Hardwick', 'Freestyle', 1500, 1912, 3), ('Warren Paoa Kealoha', 'Backstroke', 100, 1920, 1), ('Raymond Kegeris', 'Backstroke', 100, 1920, 2

## Exercice 2 : Moyenne des Résultats par Équipe

Utilisez une fonction de fenêtre pour calculer le temps moyen des résultats pour chaque équipe à travers tous les événements et listez les équipes dont la performance est supérieure à la moyenne générale.

Pour faire cette comparaison, il faut comparer type d'épreuve par type d'épreuve, et donc pour chaque type d'épreuve, calculer la moyenne des résultats.

On va donc GROUP BY épreuve.

Petite erreur de conception : pour chaque athlète y a une colonne "team", alors que d'une part un athlète peut avoir différentes équipes au cours du temps (un binational par exemple), et d'autre part tous les membres d'une "team" au sens actuel, donc les coéquipiers sur une épreuve, sont forcément de la même équipe nationale.

C'est corrigé, maintenant y a une table event_teams, et une table national_teams.

Pour l'exercice 2, on va donc calculer la moyenne des résultats pour chaque équipe sur chaque type d'épreuve, et on va filtrer les équipes dont la performance est supérieure à la moyenne générale.


In [48]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH AverageResultsPerEvent AS (
    SELECT
        e.stroke,
        e.distance,
        e.nb_relay,
        AVG(r.results) AS average_results
    FROM (SELECT * FROM results WHERE nullif(results, 'NaN') is not null and rank > 0) r
    INNER JOIN event_teams et ON r.event_team_id = et.event_team_id
    INNER JOIN events e ON et.event_id = e.event_id
    GROUP BY e.stroke, e.distance, e.nb_relay
),
AverageResultsPerEventAndNationalTeam AS (
    SELECT
        nt.code,
        e.stroke,
        e.distance,
        e.nb_relay,
        AVG(r.results) AS average_results
    FROM (SELECT * FROM results WHERE nullif(results, 'NaN') is not null and rank > 0) r
    INNER JOIN event_teams et ON r.event_team_id = et.event_team_id
    INNER JOIN events e ON et.event_id = e.event_id
    INNER JOIN national_teams nt ON et.national_team_id = nt.national_team_id
    GROUP BY e.stroke, e.distance, e.nb_relay, nt.code
)

SELECT a.stroke, a.nb_relay, a.distance, a.code, a.average_results, b.average_results
FROM AverageResultsPerEventAndNationalTeam a
INNER JOIN AverageResultsPerEvent b ON
    a.stroke = b.stroke
    AND a.distance = b.distance
    AND a.nb_relay = b.nb_relay
WHERE a.average_results < b.average_results
ORDER BY a.stroke, a.nb_relay, a.distance, a.average_results
""")

result = session.execute(query)
print(result.fetchall())

session.close()

[('Backstroke', 1, 100, 'ROC', 51.989999999999995, 64.30906515580742), ('Backstroke', 1, 100, 'AUT', 54.35, 64.30906515580742), ('Backstroke', 1, 100, 'POL', 55.04, 64.30906515580742), ('Backstroke', 1, 100, 'CUB', 55.25333333333333, 64.30906515580742), ('Backstroke', 1, 100, 'RUS', 56.080000000000005, 64.30906515580742), ('Backstroke', 1, 100, 'ESP', 56.41333333333333, 64.30906515580742), ('Backstroke', 1, 100, 'SWE', 56.935, 64.30906515580742), ('Backstroke', 1, 100, 'CHN', 57.082857142857144, 64.30906515580742), ('Backstroke', 1, 100, 'PUR', 57.28, 64.30906515580742), ('Backstroke', 1, 100, 'ISR', 57.635000000000005, 64.30906515580742), ('Backstroke', 1, 100, 'ITA', 58.7325, 64.30906515580742), ('Backstroke', 1, 100, 'EUN', 58.925, 64.30906515580742), ('Backstroke', 1, 100, 'ROU', 59.495000000000005, 64.30906515580742), ('Backstroke', 1, 100, 'ZIM', 59.845, 64.30906515580742), ('Backstroke', 1, 100, 'URS', 60.205555555555556, 64.30906515580742), ('Backstroke', 1, 100, 'GDR', 60.7287

## Exercice 3 : Amélioration des Performances au Fil du Temps

Employez un CTE pour organiser les données par athlète et année, puis utilisez des fonctions de fenêtre pour calculer la différence de résultats par rapport à l'année olympique précédente pour chaque athlète.

Je vais préciser l'énoncé : on prend la dernière année olympique qu'a couru un athlète sur une épreuve donnée, et on compare avec la précédente année olympique qu'il a courue s'il y en a une. Sinon, on ne le prend pas en compte.

In [65]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH LastOlympicYear AS (
    SELECT
        et.athlete_id,
        MAX(e.year) AS last_year
    FROM events e
    INNER JOIN event_teams et ON e.event_id = et.event_id
    WHERE e.stroke = :stroke
    AND e.distance = :distance
    AND e.nb_relay = :nb_relay
    GROUP BY et.athlete_id
),
PreviousOlympicYear AS (
    SELECT
        et.athlete_id,
        MAX(e.year) AS previous_year
    FROM events e
    INNER JOIN event_teams et ON e.event_id = et.event_id
    INNER JOIN LastOlympicYear l ON et.athlete_id = l.athlete_id
    WHERE e.year < l.last_year
    AND e.stroke = :stroke
    AND e.distance = :distance
    AND e.nb_relay = :nb_relay
    GROUP BY et.athlete_id
),
ResultsPerAthletePerYear AS (
    SELECT
        et.athlete_id,
        e.year,
        r.results
    FROM events e
    INNER JOIN event_teams et ON e.event_id = et.event_id
    INNER JOIN results r ON et.event_team_id = r.event_team_id
    WHERE e.stroke = :stroke
    AND e.distance = :distance
    AND e.nb_relay = :nb_relay
    AND r.rank > 0
)

SELECT
    a.name,
    l.last_year,
    r_last.results AS last_results,
    p.previous_year,
    r_prev.results AS previous_results,
    (r_last.results - r_prev.results) AS results_difference
FROM LastOlympicYear l
LEFT JOIN ResultsPerAthletePerYear r_last ON l.athlete_id = r_last.athlete_id AND l.last_year = r_last.year
LEFT JOIN PreviousOlympicYear p ON l.athlete_id = p.athlete_id
LEFT JOIN ResultsPerAthletePerYear r_prev ON p.athlete_id = r_prev.athlete_id AND r_prev.year = p.previous_year
INNER JOIN athletes a ON p.athlete_id = a.athlete_id
WHERE r_last.results IS NOT NULL AND r_prev.results IS NOT NULL
ORDER BY a.name
""")

nb_relay = 1
distance = 100
stroke = "Breaststroke"

result = session.execute(query, {"stroke": stroke, "distance": distance, "nb_relay": nb_relay})
results = result.fetchall()

for row in results:
    print(row[0])

session.close()

Adam Peaty
Adrian David Moorhouse
Agnes Kovacs
Alia Atkinson
Amanda Beard
Brendan Hansen
Brenton Rickard
Cameron Van Der Burgh
David Andrew Wilkie
Dmitri Volkov
Duncan Goodhew
Eva-Marie Hakansson
Guylaine Cloutier
Halyna Prozumenshchykova-Stepanova
Hugues Duboscq
John Frederick Hencken
Jose Sylvio Fiolo
Karoly Guttler
Kosuke Kitajima
Leisel Jones
Lilly King
Margaret Kelly-Hohmann
Mark Gangloff
Megan Quann
Nikolai Pankin
Penelope Heyns
Peter Evans
Philip John Rogers
Rebecca Soni
Roman Sloudnov
Ruta Meilutyte
Samantha Linette Riley
Sarah Poewe
Svitlana Bondarenko
Tarnee White
Victor Davis
Walter Kusch
Yuliya Efimova
